In [24]:
from vnstock import Company
from vnstock import Listing
from vnstock import Finance
company = Company(symbol = "VTO", source="TCBS")
company2 = Company(symbol = "YBG", source="VCI")
listing = Listing()
fin = finance = Finance(symbol="VTO", source="VCI")



In [15]:
import pandas as pd
# Cấu hình để hiển thị tất cả các cột mà không bị rút gọn
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [26]:
fin._get_report(mode='raw').head().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Columns: 213 entries, ticker to __typename
dtypes: float64(126), int64(85), object(2)
memory usage: 8.4+ KB


In [56]:
df = listing.symbols_by_industries()
df = df[df['symbol'] == 'YBM']
df


,symbol,organ_name,icb_name3,icb_name2,icb_name4,com_type_code,icb_code1,icb_code2,icb_code3,icb_code4
1270,YBM,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái,Khai khoáng,Tài nguyên Cơ bản,Khai khoáng,CT,1000,1700,1770,1775


In [52]:
df = listing.industries_icb()
df = df[df['icb_code'] == '2000']
df


,icb_name,en_icb_name,icb_code,level
146,Công nghiệp,Industrials,2000,1


In [55]:
listing.industries_icb()

,icb_name,en_icb_name,icb_code,level
0,Sản xuất Dầu khí,Oil & Gas Producers,0530,3
1,"Thiết bị, Dịch vụ và Phân phối Dầu khí","Oil Equipment, Services & Distribution",0570,3
2,Hóa chất,Chemicals,1350,3
3,Lâm nghiệp và Giấy,Forestry & Paper,1730,3
4,Kim loại,Industrial Metals & Mining,1750,3
...,...,...,...,...
150,Viễn thông,Telecommunications,6000,1
151,Tiện ích Cộng đồng,Utilities,7000,1
152,Tài chính,Financials,8000,1
153,Ngân hàng,Banks,8301,1


In [42]:
# Check existing industries and symbols to understand the data structure
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'config.settings.development')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from apps.stock.models import Symbol, Industry
from vnstock.api.listing import Listing

# Check existing data
print("Existing Industries:")
for industry in Industry.objects.all()[:10]:
    print(f"ID: {industry.id}, Name: {industry.name}, Level: {industry.level}")

print(f"\nTotal Industries: {Industry.objects.count()}")

print("\nExisting Symbols:")
for symbol in Symbol.objects.all()[:10]:
    print(f"Name: {symbol.name}, Exchange: {symbol.exchange}")
    industries = symbol.industries.all()
    if industries:
        print(f"  Industries: {[ind.name for ind in industries]}")

print(f"\nTotal Symbols: {Symbol.objects.count()}")

# Get sample data from vnstock to understand structure
listing_obj = Listing()
companies_df = listing_obj.all_symbols()
print(f"\nSample VNStock data (first 3 rows):")
print(companies_df.head(3))
print(f"\nColumns: {list(companies_df.columns)}")
print(f"\nTotal companies from VNStock: {len(companies_df)}")

Existing Industries:
ID: 530, Name: Sản xuất Dầu khí, Level: 3
ID: 570, Name: Thiết bị, Dịch vụ và Phân phối Dầu khí, Level: 3
ID: 1350, Name: Hóa chất, Level: 3
ID: 1730, Name: Lâm nghiệp và Giấy, Level: 3
ID: 1750, Name: Kim loại, Level: 3
ID: 1770, Name: Khai khoáng, Level: 3
ID: 2350, Name: Xây dựng và Vật liệu, Level: 3
ID: 2720, Name: Hàng công nghiệp, Level: 3
ID: 2730, Name: Điện tử & Thiết bị điện, Level: 3
ID: 2750, Name: Công nghiệp nặng, Level: 3

Total Industries: 131

Existing Symbols:
Name: YBM, Exchange: HSX
  Industries: ['Sản xuất Dầu khí', 'Thiết bị, Dịch vụ và Phân phối Dầu khí', 'Hóa chất', 'Lâm nghiệp và Giấy', 'Kim loại', 'Khai khoáng', 'Xây dựng và Vật liệu', 'Hàng công nghiệp', 'Điện tử & Thiết bị điện', 'Công nghiệp nặng', 'Vận tải', 'Tư vấn & Hỗ trợ Kinh doanh', 'Ô tô và phụ tùng', 'Bia và đồ uống', 'Sản xuất thực phẩm', 'Hàng gia dụng', 'Hàng hóa giải trí', 'Hàng cá nhân', 'Thuốc lá', 'Thiết bị và Dịch vụ Y tế', 'Dược phẩm', 'Phân phối thực phẩm & dược phẩm'

In [45]:
# Check VNStock API methods to understand how to get industry data
from django.db import models

# Check available VNStock methods
listing_obj = Listing()
print("Available Listing methods:")
methods = [method for method in dir(listing_obj) if not method.startswith('_')]
print(methods)

# Check different data sources
print("\n" + "="*50)
print("Testing all_symbols():")
companies_df = listing_obj.all_symbols()
print(f"Columns: {list(companies_df.columns)}")
print("Sample data:")
print(companies_df.head(3))

print("\n" + "="*50)
print("Testing symbols_by_exchange():")
try:
    hose_df = listing_obj.symbols_by_exchange('HOSE')
    print(f"HOSE Columns: {list(hose_df.columns)}")
    print("Sample HOSE data:")
    print(hose_df.head(2))
except Exception as e:
    print(f"Error getting HOSE data: {e}")

print("\n" + "="*50)
print("Testing symbols_by_group():")
try:
    group_df = listing_obj.symbols_by_group('VN30')
    print(f"VN30 Columns: {list(group_df.columns)}")
    print("Sample VN30 data:")
    print(group_df.head(2))
except Exception as e:
    print(f"Error getting VN30 data: {e}")

print("\n" + "="*50)
print("Testing industries_icb():")
try:
    industries_df = listing_obj.industries_icb()
    print(f"Industries ICB Columns: {list(industries_df.columns)}")
    print("Sample industries data:")
    print(industries_df.head(3))
except Exception as e:
    print(f"Error getting industries ICB data: {e}")

Available Listing methods:
['all_bonds', 'all_covered_warrant', 'all_future_indices', 'all_government_bonds', 'all_symbols', 'history', 'industries_icb', 'source', 'symbol', 'symbols_by_exchange', 'symbols_by_group', 'symbols_by_industries']

Testing all_symbols():
Columns: ['symbol', 'organ_name']
Sample data:
  symbol                                                 organ_name
0    YTC  Công ty Cổ phần Xuất nhập khẩu Y tế Thành phố Hồ Chí Minh
1    YEG                             Công ty Cổ phần Tập đoàn Yeah1
2    YBM             Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái

Testing symbols_by_exchange():
Error getting HOSE data: RetryError[<Future at 0x1a95f9d4b10 state=finished raised ValueError>]

Testing symbols_by_group():
Error getting VN30 data: 'Series' object has no attribute 'columns'

Testing industries_icb():
Industries ICB Columns: ['icb_name', 'en_icb_name', 'icb_code', 'level']
Sample industries data:
                                 icb_name  \
0                    

In [47]:
# Sửa đúng cách import Symbol-Industry relationships
print("Đang xóa dữ liệu bảng trung gian cũ...")

# Xóa tất cả mối quan hệ many-to-many cũ
for symbol in Symbol.objects.all():
    symbol.industries.clear()

print("Đã xóa xong dữ liệu cũ")

def correct_import():
    """Import đúng cách Symbol-Industry relationships"""
    listing_obj = Listing()
    
    # Lấy tất cả symbols với industry codes của chúng
    symbols_with_industries = listing_obj.symbols_by_industries()
    print(f"Tìm thấy {len(symbols_with_industries)} symbols với industry data")
    print(f"Columns: {list(symbols_with_industries.columns)}")
    
    # Lấy danh sách tất cả industries
    industries_df = listing_obj.industries_icb()
    print(f"Tìm thấy {len(industries_df)} industries")
    
    # Tạo dictionary để lookup industry nhanh
    industry_dict = {}
    for _, row in industries_df.iterrows():
        industry_code = row['icb_code']
        industry_name = row['icb_name']
        level = row['level']
        
        # Tạo hoặc lấy industry
        industry, created = Industry.objects.get_or_create(
            id=industry_code,
            defaults={
                'name': industry_name,
                'level': level
            }
        )
        industry_dict[industry_code] = industry
        if created:
            print(f"Tạo mới industry: {industry_name} ({industry_code})")
    
    count = 0
    # Xử lý từng symbol
    for _, row in symbols_with_industries.iterrows():
        symbol_code = row['symbol']
        if not symbol_code or str(symbol_code) == 'nan':
            continue
            
        symbol_code = str(symbol_code).upper().strip()
        
        # Tạo hoặc lấy symbol
        symbol, created = Symbol.objects.get_or_create(
            name=symbol_code,
            defaults={'exchange': 'VN'}
        )
        
        if created:
            print(f"Tạo mới symbol: {symbol_code}")
        
        # Lấy tất cả icb_codes cho symbol này
        icb_columns = ['icb_code1', 'icb_code2', 'icb_code3', 'icb_code4']
        
        for col in icb_columns:
            if col in row and pd.notna(row[col]):
                icb_code = int(row[col])
                
                if icb_code in industry_dict:
                    industry = industry_dict[icb_code]
                    # Thêm industry vào symbol
                    symbol.industries.add(industry)
                    count += 1
                    
                    if count % 100 == 0:
                        print(f"Đã xử lý {count} mối quan hệ...")
                else:
                    print(f"Không tìm thấy industry với code {icb_code}")
    
    print(f"Hoàn thành! Tạo {count} mối quan hệ")
    
    # Kiểm tra kết quả
    print(f"Tổng symbols: {Symbol.objects.count()}")
    print(f"Tổng industries: {Industry.objects.count()}")
    
    # Hiển thị mẫu cho symbol VTO
    print("\nKiểm tra symbol VTO:")
    try:
        vto = Symbol.objects.get(name='VTO')
        industries = [f"{ind.name} ({ind.id})" for ind in vto.industries.all()]
        print(f"VTO có {len(industries)} industries: {industries}")
    except Symbol.DoesNotExist:
        print("Không tìm thấy symbol VTO")
    
    # Hiển thị thêm mẫu
    print("\nMẫu kết quả khác:")
    for symbol in Symbol.objects.prefetch_related('industries')[:5]:
        industries = [f"{ind.name} ({ind.id})" for ind in symbol.industries.all()]
        if industries:
            print(f"{symbol.name}: {len(industries)} industries - {', '.join(industries[:2])}")

correct_import()

Starting import process...
Found 155 industries in VNStock
Updated industry: Sản xuất Dầu khí (Code: 0530)
Error getting symbols for industry 0530 (Sản xuất Dầu khí): RetryError[<Future at 0x1a95b47ddd0 state=finished raised ValueError>]
Updated industry: Thiết bị, Dịch vụ và Phân phối Dầu khí (Code: 0570)
Error getting symbols for industry 0570 (Thiết bị, Dịch vụ và Phân phối Dầu khí): RetryError[<Future at 0x1a96047ecd0 state=finished raised ValueError>]
Updated industry: Hóa chất (Code: 1350)
Error getting symbols for industry 1350 (Hóa chất): RetryError[<Future at 0x1a96047c790 state=finished raised ValueError>]
Updated industry: Lâm nghiệp và Giấy (Code: 1730)
Error getting symbols for industry 1730 (Lâm nghiệp và Giấy): RetryError[<Future at 0x1a95df7a710 state=finished raised ValueError>]
Updated industry: Kim loại (Code: 1750)
Error getting symbols for industry 1750 (Kim loại): RetryError[<Future at 0x1a96060aed0 state=finished raised ValueError>]
Updated industry: Khai khoáng 

KeyboardInterrupt: 

In [48]:
# Kiểm tra kết quả import cho symbol VTO
print("Kiểm tra symbol VTO sau khi import:")

try:
    vto = Symbol.objects.get(name='VTO')
    industries = vto.industries.all()
    
    print(f"Symbol VTO có {industries.count()} industries:")
    for industry in industries:
        print(f"  - {industry.name} (ID: {industry.id})")
    
    # So sánh với data gốc từ VNStock
    print(f"\nSo sánh với VNStock data:")
    symbols_df = listing_obj.symbols_by_industries()
    vto_row = symbols_df[symbols_df['symbol'] == 'VTO']
    
    if not vto_row.empty:
        row = vto_row.iloc[0]
        vnstock_codes = []
        for col in ['icb_code1', 'icb_code2', 'icb_code3', 'icb_code4']:
            if col in row and pd.notna(row[col]):
                vnstock_codes.append(int(row[col]))
        
        print(f"VNStock codes cho VTO: {vnstock_codes}")
        
        # Kiểm tra xem có khớp không
        db_codes = [ind.id for ind in industries]
        missing_codes = set(vnstock_codes) - set(db_codes)
        extra_codes = set(db_codes) - set(vnstock_codes)
        
        if missing_codes:
            print(f"Thiếu codes: {missing_codes}")
        if extra_codes:
            print(f"Thừa codes: {extra_codes}")
        if not missing_codes and not extra_codes:
            print("✅ Perfect match!")
    
except Symbol.DoesNotExist:
    print("❌ Không tìm thấy symbol VTO trong database")

# Kiểm tra thêm vài symbols khác
print(f"\n" + "="*50)
print("Kiểm tra thêm một số symbols khác:")

for symbol in Symbol.objects.prefetch_related('industries')[:3]:
    industries_count = symbol.industries.count()
    if industries_count > 0:
        industries_names = [ind.name for ind in symbol.industries.all()]
        print(f"{symbol.name}: {industries_count} industries - {', '.join(industries_names[:2])}{'...' if len(industries_names) > 2 else ''}")

Kiểm tra symbol VTO sau khi import:
Symbol VTO có 131 industries:
  - Sản xuất Dầu khí (ID: 530)
  - Thiết bị, Dịch vụ và Phân phối Dầu khí (ID: 570)
  - Hóa chất (ID: 1350)
  - Lâm nghiệp và Giấy (ID: 1730)
  - Kim loại (ID: 1750)
  - Khai khoáng (ID: 1770)
  - Xây dựng và Vật liệu (ID: 2350)
  - Hàng công nghiệp (ID: 2720)
  - Điện tử & Thiết bị điện (ID: 2730)
  - Công nghiệp nặng (ID: 2750)
  - Vận tải (ID: 2770)
  - Tư vấn & Hỗ trợ Kinh doanh (ID: 2790)
  - Ô tô và phụ tùng (ID: 3350)
  - Bia và đồ uống (ID: 3530)
  - Sản xuất thực phẩm (ID: 3570)
  - Hàng gia dụng (ID: 3720)
  - Hàng hóa giải trí (ID: 3740)
  - Hàng cá nhân (ID: 3760)
  - Thuốc lá (ID: 3780)
  - Thiết bị và Dịch vụ Y tế (ID: 4530)
  - Dược phẩm (ID: 4570)
  - Phân phối thực phẩm & dược phẩm (ID: 5330)
  - Bán lẻ (ID: 5370)
  - Truyền thông (ID: 5550)
  - Du lịch & Giải trí (ID: 5750)
  - Viễn thông cố định (ID: 6530)
  - Viễn thông di động (ID: 6570)
  - Sản xuất & Phân phối Điện (ID: 7530)
  - Nước & Khí đốt (ID